In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
# Display dataset shapes
print("Training images shape:", train_images.shape)
print("Training labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)
print("Sample training label:", train_labels[0])
print("Sample training image shape:", train_images[0].shape)

In [ ]:
# Get unique labels
import numpy as np

labels = np.unique(train_labels)
print("labels are:", labels)

In [ ]:
# Function to plot sample images
import matplotlib.pyplot as plt

def plot_sample_images(images, labels, num_samples=5):
    plt.figure(figsize=(10, 2))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(images[i], cmap='gray')
        plt.title(f"Label: {labels[i]}")
        plt.axis('off')
    plt.show()
    
plot_sample_images(train_images, train_labels)

In [ ]:
# Label distribution plot
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.countplot(x=train_labels.tolist())
plt.title("Distribution of Labels in Training Set")
plt.xlabel("Label")
plt.ylabel("Count")
plt.show()

In [ ]:
# Function to plot images by class
def plot_images_by_class(images, labels, num_classes=10, num_samples=5):
    plt.figure(figsize=(15, num_classes * 3))
    for label in range(num_classes):
        class_images = images[np.where(labels == label)]
        for i in range(num_samples):
            plt.subplot(num_classes, num_samples, label * num_samples + i + 1)
            plt.imshow(class_images[i], cmap='gray')
            plt.title(f"Label: {label}")
    plt.show()
    
plot_images_by_class(train_images, train_labels)

In [ ]:
# Normalize images
from tensorflow.keras.utils import to_categorical

train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

# Convert labels to categorical
train_labels_categorical = to_categorical(train_labels)
test_labels_categorical = to_categorical(test_labels)

Model 1: Long Short-Term Memory

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout, Conv2D, MaxPooling2D, Flatten

model_lstm = Sequential([
    Input(shape=(28, 28)),
    LSTM(128, return_sequences=True),
    Dropout(0.1),
    LSTM(64),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_lstm.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

callbacks = [ModelCheckpoint('digit_recognition_model.keras', verbose=1, save_best_only=True)]

In [ ]:
history_lstm = model_lstm.fit(train_images, train_labels_categorical, epochs=15, batch_size=128, validation_data=(test_images, test_labels_categorical), callbacks=callbacks)

In [ ]:
# Evaluate LSTM model
test_loss, test_accuracy = model_lstm.evaluate(test_images, test_labels_categorical)
print('Test accuracy:', test_accuracy)

In [ ]:
# Training history visualization
def plot_training_history(history, title):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title(title + ' Accuracy')
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(title + ' Loss')
    plt.show()

In [ ]:
plot_training_history(history_lstm, "LSTM")

In [ ]:
def predict_sample(model, sample_image, title):
    sample_image = np.expand_dims(sample_image, axis=0)
    prediction = model.predict(sample_image)
    predicted_class = np.argmax(prediction)
    
    plt.imshow(sample_image[0], cmap='gray')
    plt.title(f"Predicted Class ({title}): {predicted_class}")
    plt.show()

In [ ]:
predict_sample(model_lstm, test_images[24], "LSTM")

Model 2: Convolutional Neural Network

In [ ]:
train_images_cnn = train_images.reshape(-1, 28, 28, 1)
test_images_cnn = test_images.reshape(-1, 28, 28, 1)

In [ ]:
model_cnn = Sequential([
    Input(shape=(28, 28, 1)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model_cnn.compile(optimizer= 'adam', loss= 'categorical_crossentropy', metrics= ['accuracy'])
model_cnn.summary()

In [ ]:
history_cnn = model_cnn.fit(train_images_cnn, train_labels_categorical, epochs=15, batch_size=128, validation_data=(test_images_cnn, test_labels_categorical), callbacks=callbacks)

In [ ]:
# Evaluate CNN model
test_loss, test_accuracy = model_cnn.evaluate(test_images_cnn, test_labels_categorical)
print('CNN Test accuracy:', test_accuracy)

In [ ]:
plot_training_history(history_cnn, "CNN")

In [ ]:
predict_sample(model_cnn, test_images_cnn[24], "CNN")

Model 3: Random Forest Classifier

In [ ]:
train_images_flat = train_images.reshape(len(train_images), -1)
test_images_flat = test_images.reshape(len(test_images), -1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_images_flat, train_labels, test_size=0.3, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42)
model_rf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Random Forest Accuracy:", accuracy)

In [ ]:
sample_image_rf = test_images[24].reshape(1, -1)
predicted_image_rf = model_rf.predict(sample_image_rf)[0]

plt.imshow(test_images[24], cmap='gray')
plt.title(f"Predicted Class (Random Forest): {predicted_image_rf}")
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Confusion Matrix for Random Forest
conf = confusion_matrix(y_test, y_pred)
display = ConfusionMatrixDisplay(confusion_matrix=conf)
display.plot()
plt.title("Confusion Matrix of Random Forest ")
plt.show()

In [ ]:
# Classification Report
from sklearn.metrics import classification_report

print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred))